In [1]:
import os
import json
import random
import requests
import time
from dotenv import load_dotenv
from volcenginesdkarkruntime import Ark
from llm_utils import llm_gen_conversation_summary, llm_gen_memory

# 加载环境变量
load_dotenv()

class LLMService:
    def __init__(self):
        self.model_client = Ark(
            base_url="https://ark.cn-beijing.volces.com/api/v3",
            api_key=os.getenv("ARK_API_KEY"),
        )
        self.conf = {
            "model_name": "doubao-seed-1-6-250615",
            "max_tokens": 32768,
        }

    def call_openai_api_with_retry(self, messages, max_retries=2, delay=60):
        retries = 0
        while retries < max_retries:
            try:
                # if self.model == 'doubao-1.6':
                #     response = self.model_client.chat.completions.create(model=self.conf["model_name"], messages=messages, max_tokens=self.conf["max_tokens"], thinking={"type":"disabled"})
                # else:
                response = self.model_client.chat.completions.create(model=self.conf["model_name"], messages=messages)
                return response
            # except openai.OpenAIError as e:
            #     print(f"OpenAI error occurred: {e}")
            except Exception as e:
                print(f"An error occurred: {e}")
            retries += 1
            _delay = delay + random.randint(1, 5) * delay
            print(f"Retrying... ({retries}/{max_retries}), sleep: {_delay}s")
            time.sleep(_delay)
        raise Exception("API call failed after maximum retries")

    def chat(self, system_prompt, prompt):
        messages = []
        if len(system_prompt) > 0:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": prompt})
        try:
            response = self.call_openai_api_with_retry(messages)
            call_llm_log = {
                "model": self.conf["model_name"],
                "messages": messages,
                "response": response.choices[0].message.content.strip()
            }
            with open('call_llm_log.json', 'a') as f:
                json.dump(call_llm_log, f, ensure_ascii=False)
                f.write('\n')
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Final error: {e}")
            return ''

    def generate_summary(self, conversation_content):
        return llm_gen_conversation_summary(self, conversation_content)

    def generate_memory(self, historical_data, latest_day_data):
        return llm_gen_memory(self, historical_data, latest_day_data)


llm_service = LLMService()


if __name__ == "__main__":
    llm_service = LLMService()
    print(llm_service.chat("你是一个助手", "你好"))

/opt/anaconda3/envs/delta_web/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


你好！很高兴能和你打招呼，有什么我可以帮到你的吗？😊


In [3]:
content_day1 = '''
6.1 06:30
妻子：[语音留言，背景有豆浆机滴滴提示音]“老公啊，豆浆机我昨晚就预约好了，今早就有热豆浆喝。你千万记得把玄关那个样品芯片盒带走，我昨天收拾客厅的时候，看见落茶几上了，可别又忘带了啊。”
李哲：[文字回复]“知道了。晚上咱们去鼎泰丰吃饭怎么样？儿子的奥数课我去接他，你就不用操心了。”

6.1 07:45
邻居王总：“李总，又去跑半马了呀？我听说你们最近投了燧原科技的新架构芯片，是不是啊？”
李哲：“（喘气声，布料摩擦声）对啊，刚跑完。测试数据出来了，性能超过 A100 三成呢。不过现在量产遇到麻烦了，良率卡在 58% 上不去，老张他们团队这个月得全住进台积电去攻关了。”

6.1 09:15
分析师小林：“李总，墨丘利医疗的 CTO 昨天夜里离职了，我把竞业协议的疑点清单发到您邮箱了，您看看。”
李哲：“三点之前给我把法律尽调报告整理好。还有啊，（桌面敲击声）那家大模型公司别再给我推荐了，他们的 token 成本居然是 OpenAI 的 7 倍，这也太离谱了！”

6.1 10:40
思睿机器人 CEO：“李董，不好了，波士顿动力刚刚起诉我们，说我们关节专利侵权了...”
李哲：“（纸张翻动声）别慌，用 2019 年浙大那篇论文里的算法就能规避，我马上让金杜的王律师半小时内联系你，他会处理好的。”

6.2 12:10
政府引导基金负责人：“李总，第三届人工智能岛的税收优惠细则下周就要出台了，你们的硬科技基金要是落地浦东，有一些附加条件需要了解一下...”
李哲：（餐具碰撞声）“我关心的关键点就两个：设备进口关税减免能不能覆盖研发阶段？还有海外人才的个税补贴，能不能突破 15% 的上限？”

6.2 14:00
脑机接口创业者：“李总，我们的动物实验进展不错，LFP 信号解码准确率达到 91% 了...”
李哲：“把数据包发给我，要原始格式的。（几秒钟停顿）我看了你们的报告，第三组猕猴的颅内感染率有 23%，这样的话，FDA 的临床批文你们打算怎么拿下来？”

6.2 15:30
儿子班主任：“各位家长请注意，请还未提交疫苗接种证明的家长私信我一下。”
李哲：[语音转文字，背景有嘈杂人声]“刘老师不好意思啊，接种证明文件在我助理那里，我稍后让她联系您。我现在正在参加半导体峰会呢。”

6.3 17:00
合伙人 A：“李总，这个 AIGC 团队开价太高了，要 25 亿的估值！”
李哲：“他们说的‘百亿级中文语料’，实际清洗之后不足 30T。（电子设备操作声）你看数据面板这里，测试集里有 42% 的内容都是从知乎盗爬过来的。”

6.3 19:40
儿子：“爸！我的机器人比赛马上要用激光雷达，你快帮我想想办法啊...”
李哲：（手机摄像头对焦声）“你去申请这个开放实验室的账号，他们有 Ouster 最新的 OS1 传感器，性能特别好。（转向背景）老婆，我下周可能要去无锡两天，中芯国际那条产线有点问题得去看看。”

6.5 21:15
某 FA：“紧急消息！某自动驾驶独角兽 pre - IPO 份额有 5000 万刀的额度，要不要考虑一下？”
李哲：“大家注意啊，他们 Q1 的毛利率是 - 63%，各位务必仔细核查一下，看看他们的电池租赁资产有没有被重复抵押。”

6.5 23:00
妻子：“老公，你要吃的药在床头柜第二格。（短暂停顿）妈打电话说，养老院的 AI 照护系统半夜总是亮蓝光，影响她休息。”
李哲：（键盘敲击声）“我让我们投资的那家企业明天寄点遮光滤片过去。（叹气声）这个项目社会价值评分倒是打满了，就是产品经理太缺乏适老化思维了，得好好改进一下。”
'''

content_day2 = '''
6.6 07:15
台积电工程师：“李总，新制程试产良率提到 62%，但光刻胶残留问题导致 3 纳米线路偏移。”
李哲：“（翻阅文件声）把 ASML 最新校准参数同步给材料组，联系日本合成橡胶株式会社，要求他们 48 小时内提供低黏度胶液方案。”
6.6 09:00
医疗投资总监：“李总，收购墨丘利医疗的报价被拒，对方要求溢价 30%。”
李哲：“（敲击计算器声）告诉他们，若接受对赌协议 —— 三年内实现 15% 市占率，我们可追加 5% 股权。另外，密切关注他们新离职 CTO 的去向。”
6.6 11:20
养老院院长：“李先生，您母亲用上遮光滤片后睡眠好多了，但 AI 照护系统又频繁误报跌倒警报。”
李哲：“（语速加快）让研发团队调取近一周误报数据，重点分析午饭后和凌晨时段，必须在三天内优化算法。”
6.6 13:45
自动驾驶独角兽 CFO：“李总，关于资产抵押质疑，我们提供了第三方审计报告...”
李哲：“（打断）报告里 2024 年 Q4 的电池资产折旧率与行业标准不符，你们必须解释清楚。另外，我需要与你们 CTO 单独沟通技术路线。”
6.6 15:30
浙大教授：“小李，你们用我们论文算法规避专利的事，校方希望能展开产学研合作。”
李哲：“求之不得！我们可以共建实验室，研发经费校方占比 40%，成果共享。具体条款我让法务明天就拟好。”
6.6 17:10
儿子同学家长：“李哲爸爸，机器人比赛需要家长志愿者，您能抽空来帮忙吗？”
李哲：[语音转文字，背景有会议讨论声]“实在抱歉，最近项目太忙了。但我可以赞助些传感器设备，需要的话随时联系我助理。”
6.6 19:30
妻子：“老公，儿子机器人比赛进决赛了！不过他说需要更高级的控制器。”
李哲：（翻找抽屉声）“我记得书房有个闲置的树莓派 4B，明天让他带上。对了，我后天出发去无锡，你帮我收拾下行李？”
6.6 22:00
被投大模型公司创始人：“李总，我们优化了 token 机制，成本降低到 OpenAI 的 4 倍了！”
李哲：“（冷笑）先解决数据版权问题再说。我刚收到举报，你们训练集里有大量未经授权的学术论文。”
6.6 23:30
金杜王律师：“李总，思睿机器人的专利诉讼有转机，波士顿动力关键证据链存在瑕疵。”
李哲：“太好了！乘胜追击，反诉他们滥用诉权。明天上午十点，我们开个线上会议详细部署。”
'''

summary = llm_service.generate_summary(content)
print(summary)

{
  "summary": "李哲在6月1日至5日期间，与妻子、邻居、同事、合作伙伴等多人就工作事务（如芯片量产、投资项目、法律纠纷等）和家庭生活（如接送孩子、家庭聚餐、母亲照护等）进行了多轮交流。",
  "topics": [
    {
      "title": "家庭生活安排",
      "summary": "妻子提醒李哲带样品芯片盒，李哲提议晚上去鼎泰丰吃饭并负责接儿子奥数课，后又告知妻子下周去无锡出差，以及处理母亲养老院AI照护系统蓝光问题。",
      "information": "妻子预约了豆浆机，提醒李哲带走玄关的样品芯片盒；李哲提议晚上去鼎泰丰吃饭，负责接儿子奥数课；李哲告知妻子下周去无锡两天处理中芯国际产线问题；母亲反映养老院AI照护系统半夜亮蓝光影响休息，李哲让投资企业寄遮光滤片并指出产品经理缺乏适老化思维。",
      "type": "家庭生活",
      "sentiment": "中性"
    },
    {
      "title": "燧原科技芯片项目",
      "summary": "邻居询问李哲投资的燧原科技新架构芯片情况，李哲称性能超过A100三成，但量产良率卡在58%，团队需住进台积电攻关。",
      "information": "李哲他们投了燧原科技的新架构芯片，测试数据显示性能超过A100三成，目前量产遇到麻烦，良率卡在58%，老张团队这个月得全住进台积电去攻关。",
      "type": "工作事务",
      "sentiment": "消极"
    },
    {
      "title": "墨丘利医疗竞业协议及大模型公司评估",
      "summary": "分析师小林告知李哲墨丘利医疗CTO离职并发送竞业协议疑点清单，李哲要求三点前整理好法律尽调报告，并拒绝推荐token成本是OpenAI7倍的大模型公司。",
      "information": "墨丘利医疗的CTO昨天夜里离职，分析师小林把竞业协议的疑点清单发到李哲邮箱；李哲要求小林三点之前把法律尽调报告整理好；李哲因某大模型公司token成本是OpenAI的7倍而拒绝再被推荐该公司。",
      "type": "工作事务",
      "sentiment": "中性"
